In [15]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from PIL import Image
import json

# 경로 설정
training_image_path = "/home/gyuha_lee/DCC2024/dataset/bg_remove/rembg/90/training_image_no_bg"
validation_image_path = "/home/gyuha_lee/DCC2024/dataset/bg_remove/rembg/90/validation_image_no_bg"
model_path = "/home/gyuha_lee/DCC2024/mission1/WITHOUT_PRETRAINED/resnet18_gender_style_pretrained.pth"
json_path = "/home/gyuha_lee/DCC2024/mission2/top_100_preference.json"

# ResNet-18 모델 로드 및 학습된 가중치 불러오기
model = models.resnet18(pretrained=False)
model.fc = nn.Identity()  # 마지막 FC 레이어를 제거하여 중간 레이어의 feature vector를 추출하도록 설정
model.load_state_dict(torch.load(model_path), strict=False)  # 가중치 로드
model.eval()  # 모델을 평가 모드로 전환

# 이미지 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 특징 벡터 추출 함수 정의
def extract_feature_vector(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 배치 차원 추가
    with torch.no_grad():
        feature_vector = model(image).squeeze().numpy()  # 특징 벡터 추출
    return feature_vector


/home/gyuha_lee/miniconda3/envs/dcc2024/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gyuha_lee/miniconda3/envs/dcc2024/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_169352/3860674976.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weigh

In [16]:
# 특징 벡터 추출 함수 정의
def extract_feature_vector(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 배치 차원 추가
    with torch.no_grad():
        feature_vector = model(image).squeeze().numpy()  # 특징 벡터 추출
    return feature_vector

# 모든 Training 이미지의 feature vector를 미리 추출하고 저장
training_features = []
training_labels = []

with open(json_path, 'r') as f:
    data = json.load(f)

# 2번 과정: Training 데이터의 특징 벡터 추출 및 저장
user_counter = 0
for user_id, user_data in data.items():
    user_counter += 1
    print(f"사용자 {user_counter}/{len(data)}의 Training 이미지 특징 벡터 추출 중...")
    
    # 선호 및 비선호 이미지 리스트
    training_preferred_files = user_data['Training']['선호']
    training_non_preferred_files = user_data['Training']['비선호']

    # 각 이미지를 통해 특징 벡터 추출 및 저장
    for img in training_preferred_files:
        feature = extract_feature_vector(os.path.join(training_image_path, img))
        training_features.append((feature, 1))  # (특징 벡터, 선호 레이블) 형태로 저장

    for img in training_non_preferred_files:
        feature = extract_feature_vector(os.path.join(training_image_path, img))
        training_features.append((feature, 0))  # (특징 벡터, 비선호 레이블) 형태로 저장

print("모든 Training 이미지의 특징 벡터 추출 완료.")


사용자 1/100의 Training 이미지 특징 벡터 추출 중...
사용자 2/100의 Training 이미지 특징 벡터 추출 중...
사용자 3/100의 Training 이미지 특징 벡터 추출 중...
사용자 4/100의 Training 이미지 특징 벡터 추출 중...
사용자 5/100의 Training 이미지 특징 벡터 추출 중...
사용자 6/100의 Training 이미지 특징 벡터 추출 중...
사용자 7/100의 Training 이미지 특징 벡터 추출 중...
사용자 8/100의 Training 이미지 특징 벡터 추출 중...
사용자 9/100의 Training 이미지 특징 벡터 추출 중...
사용자 10/100의 Training 이미지 특징 벡터 추출 중...
사용자 11/100의 Training 이미지 특징 벡터 추출 중...
사용자 12/100의 Training 이미지 특징 벡터 추출 중...
사용자 13/100의 Training 이미지 특징 벡터 추출 중...
사용자 14/100의 Training 이미지 특징 벡터 추출 중...
사용자 15/100의 Training 이미지 특징 벡터 추출 중...
사용자 16/100의 Training 이미지 특징 벡터 추출 중...
사용자 17/100의 Training 이미지 특징 벡터 추출 중...
사용자 18/100의 Training 이미지 특징 벡터 추출 중...
사용자 19/100의 Training 이미지 특징 벡터 추출 중...
사용자 20/100의 Training 이미지 특징 벡터 추출 중...
사용자 21/100의 Training 이미지 특징 벡터 추출 중...
사용자 22/100의 Training 이미지 특징 벡터 추출 중...
사용자 23/100의 Training 이미지 특징 벡터 추출 중...
사용자 24/100의 Training 이미지 특징 벡터 추출 중...
사용자 25/100의 Training 이미지 특징 벡터 추출 중...
사용자 26/100의 Training 이미지 특징 벡터 추출 

In [17]:
# 3번 과정: Validation 이미지에 대해 KNN 예측 수행
K = 5  # K 값 설정

results = []
user_counter = 0
for user_id, user_data in data.items():
    user_counter += 1
    print(f"사용자 {user_counter}/{len(data)}의 Validation 이미지 예측 중...")
    
    validation_files = user_data['Validation']['선호'] + user_data['Validation']['비선호']

    for val_img in validation_files:
        # 1. Validation 이미지의 특징 벡터 추출
        val_feature = extract_feature_vector(os.path.join(validation_image_path, val_img))

        # 2. 모든 Training 이미지와의 유사도 계산 (Validation 이미지 특징 벡터 사용)
        similarities = [cosine_similarity(val_feature.reshape(1, -1), train_feat.reshape(1, -1))[0][0] for train_feat, _ in training_features]

        # 3. 가장 유사한 K개의 인덱스 찾기
        top_k_indices = np.argsort(similarities)[-K:]

        # 4. K개의 가장 유사한 이미지들의 선호 여부를 통해 예측
        top_k_labels = [training_features[i][1] for i in top_k_indices]
        top_k_similarities = [similarities[i] for i in top_k_indices]

        # 가중치를 적용한 예측
        weighted_sum = sum(label * sim for label, sim in zip(top_k_labels, top_k_similarities))
        predicted_label = 1 if weighted_sum >= (sum(top_k_similarities) / 2) else 0

        true_label = 1 if val_img in user_data['Validation']['선호'] else 0

        # 결과 저장
        results.append({
            'user_id': user_id,
            'validation_file': val_img,
            'predicted_label': '선호' if predicted_label == 1 else '비선호',
            'true_label': '선호' if true_label == 1 else '비선호'
        })

        # 실시간 검증 결과 출력
        print(f"검증 결과 - 사용자: {user_id}, 파일: {val_img}, 예측: {'선호' if predicted_label == 1 else '비선호'}, 실제: {'선호' if true_label == 1 else '비선호'}")

print("모든 Validation 이미지에 대한 예측 완료.")


사용자 1/100의 Validation 이미지 예측 중...
검증 결과 - 사용자: 64747, 파일: W_46907_80_powersuit_W.jpg, 예측: 선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_44330_10_sportivecasual_W.jpg, 예측: 선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_39164_00_oriental_W.jpg, 예측: 비선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_37491_70_military_W.jpg, 예측: 선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_20598_70_military_W.jpg, 예측: 선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_30988_90_kitsch_W.jpg, 예측: 선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_38588_19_genderless_W.jpg, 예측: 비선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_22510_80_powersuit_W.jpg, 예측: 선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_05628_00_cityglam_W.jpg, 예측: 비선호, 실제: 선호
검증 결과 - 사용자: 64747, 파일: W_34024_10_sportivecasual_W.jpg, 예측: 비선호, 실제: 비선호
검증 결과 - 사용자: 64747, 파일: W_14102_50_feminine_W.jpg, 예측: 비선호, 실제: 비선호
검증 결과 - 사용자: 64747, 파일: W_47169_70_hippie_W.jpg, 예측: 선호, 실제: 비선호
검증 결과 - 사용자: 64747, 파일: W_02498_50_feminine_W.jpg, 예측: 비선호, 실제: 비선호
검증 결과 - 사용자: 64747, 파일: W_11610_90_grunge_W.jpg, 예측: 비선호, 실제: 비선호
검증 결과 - 사용자: 64747, 

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# 예측 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)
true_labels = results_df['true_label'].map({'선호': 1, '비선호': 0})
predicted_labels = results_df['predicted_label'].map({'선호': 1, '비선호': 0})

# 성능 지표 계산
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# 성능 결과 출력
print("----- Model Performance Metrics -----")
print(f"Accuracy (정확도): {accuracy:.2f}")
print(f"Precision (정밀도): {precision:.2f}")
print(f"Recall (재현율): {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("-------------------------------------")


----- Model Performance Metrics -----
Accuracy (정확도): 0.67
Precision (정밀도): 0.60
Recall (재현율): 0.51
F1 Score: 0.55
-------------------------------------
